Install and import dependencies

In [ ]:
!pip install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

Helper Functions

In [ ]:
#returns True if the one if the elements in inlist is in the string s
def StringinList(inlist, s):
  for item in inlist:
    if item.lower() in s.lower():
      return True
  return False

#extracts hastags from a string
def GetHashtags(inString):
  outlist=[]
  ss=instring.split()
  for s in ss:
    if s.startswith("#"):
      outlist.append(s)
  return outlist

#appends the tweet to the attribute container
def AppendToContainer(incontainer, tweet):
  incontainer.append([tweet.id, tweet.conversationId, tweet.user.id, tweet.user.rawDescription, tweet.user.followersCount, 
                                 tweet.user.friendsCount, 
                                 tweet.user.location, tweet.retweetCount, tweet.viewCount, 
                                 tweet.date.timestamp(),tweet.date.strftime('%Y-%m-%d %H:%M:%S'), 
                                 tweet.likeCount, tweet.sourceLabel, tweet.rawContent, tweet.vibe ])


In [ ]:
attributes_container=[]
numberretrieved = 0
searchterms = ["republic ", "monarch"]
initialquery = '(#AusPol) since:2021-01-01'
hashtagdict={}
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(initialquery).get_items()):
    if StringinList(searchterms, tweet.rawContent):
      attributes_container.attributes_container.AppendToContainer(tweet)
      hashtaglist=GetHashtags(tweet.rawContent)
      for h in hashtaglist:
        hl = GetHashtags(h)
        if hashtagdict.keys.contains(hl):
          hashtagdict.update({hl: hashtagdict[hl]+1}) 
        else:
          hashtagdict[hl]=1
    numberretrieved+=1
    if numberretrieved % 100 == 0:
      time.sleep(0.5)
      print(numberretrieved)
      if numberretrieved>20000:
        break
 


Add to container list based on other hashtags


In [ ]:
sortedhashtages = {k: v for k, v in sorted(hashtagdict.items(), key=lambda item: -item[1])}
listhashtags = list(sortedhashtages.keys())[0:3]
for h in listhashtags:
  query = '(' + h + ') since:2021-01-01'
  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if StringinList(searchterms, tweet.rawContent):
      attributes_container.attributes_container.AppendToContainer(tweet)

Create a Dataframe and save

In [ ]:
tweets_df = pd.DataFrame(attributes_container, columns=["ID", "ConversationID", "UserId", "UserDescription", "FollowersCount",  "FriendsCount", "Location", "Retweets", "ViewCount", "EpochTime","UTCTime" ,"Likes", "Source", "Content", "Vibe", ])
tweets_df.to_excel("/content/drive/MyDrive/A3/TweetsBackup.xlsx")

In [ ]:

#tweets_df.to_excel("/content/drive/MyDrive/A3/AusRepublic.xlsx")
tweets_df.to_excel("/content/drive/MyDrive/A3/TweetsBackup.xlsx")

Mounted at /content/drive


**Data Wrangling**

Import dependencies

In [ ]:
import pandas as pd
import nltk
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt 
import seaborn as sns
nltk.download('punkt')
nltk.download('stopwords')
#from tqdm import tqdm
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

#concatenate all workbooks
df1 = pd.read_excel("/content/drive/MyDrive/A3/AuspolRoyal.xlsx")
df2 = pd.read_excel("/content/drive/MyDrive/A3/AusRepublic.xlsx")
df3 = pd.read_excel("/content/drive/MyDrive/A3/AuspolMonarch.xlsx")
df4 = pd.read_excel("/content/drive/MyDrive/A3/ConstitutionalMonarchy.xlsx")
df5 = pd.read_excel("/content/drive/MyDrive/A3/AuspolRepublic.xlsx")
df6 = pd.read_excel("/content/drive/MyDrive/A3/VoteNoRepublic.xlsx")
df = pd.concat([df1, df2, df3, df4, df5, df6])
df.to_excel("/content/drive/MyDrive/A3/RawData.xlsx")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Initial Exploration

Helper Functions

In [ ]:
#from nltk.corpus import stopwords
stopwordsenglish = set(stopwords.words('english'))

# function to remove stopwords
def removestopwords(text, sw):
    no_stopword_text = [w for w in text.split() if not w in sw]
    return ' '.join(no_stopword_text)

def cleantext(text, sw):
    text = re.sub("\'", "", text) 
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    text = text.lower() 
    text = removestopwords(text, sw)
    return text

def freq_words(x, terms = 30): 
  all_words = ' '.join([text for text in x]) 
  all_words = all_words.split() 
  fdist = nltk.FreqDist(all_words) 
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())}) 
  
  # selecting top 20 most frequent words 
  d = words_df.nlargest(columns="count", n = terms) 
  
  # visualize words and frequencies
  plt.figure(figsize=(12,15)) 
  ax = sns.barplot(data=d, x= "count", y = "word") 
  ax.set(ylabel = 'Word') 
  plt.show()

def uniqueWords(x):
  words = ' '.join([text for text in x]) 
  words = nltk.tokenize.word_tokenize(words)
  fdist1 = nltk.FreqDist(words)
  freq = dict((word, freq) for word, freq in fdist1.items() if not word.isdigit())
  return freq


Unique entries prior to processing

In [ ]:
df.describe()
                   


In [ ]:
def tx(x):
  return str(x) + "A"

#entries
numentries= len(uniqueWords(df['Content']).keys())  
print (numentries)  #50234  
#conversations
strc = df['ConversationID'].apply(tx)
conversations = uniqueWords(strc)
numconversations = len(conversations.keys() )    
print(numconversations)     #18389  
uconv = freq_words(strc) 
describe(df)


Reload backup file

In [ ]:

df = pd.read_excel("/content/drive/MyDrive/A3/RawData.xlsx")

#clean data
print(df.shape[0]) #20501  Unc
df.drop_duplicates(subset=['ID'], inplace=True)
print(df.shape[0]) #19419
#drop the persistent index
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df['SampleResponse'] = np.nan

Clean Content

In [ ]:
df['CleanContent'] = df['Content'].apply(lambda x: cleantext(x, stopwordsenglish))

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/A3/RawDatay.xlsx")
print(df.shape[0]) #19419


19419
